In [ ]:
import pickle
import numpy as np

# Load your sb3 replay buffer
with open('/home/nitesh/workspace/offline_rl_test/replay_buffer_with_embeddings_7.pkl', 'rb') as f:
    sb3_buffer = pickle.load(f)

In [ ]:
sb3_buffer[0].shape

In [ ]:
observations = np.array(sb3_buffer[0][:, 0, :])
actions = np.array(sb3_buffer[1][:, 0, :])
rewards = np.array(sb3_buffer[2][:, 0, :])
dones = np.array(sb3_buffer[3][:, 0, :])
truncated = np.array(sb3_buffer[4][:, 0, :])
observations = np.concatenate([observations, rewards], axis=1)

In [ ]:
actions.max(), actions.min()

In [ ]:
observations.shape, rewards.shape

In [ ]:
from d3rlpy.dataset import MDPDataset
import d3rlpy

In [ ]:
mdp_dataset = MDPDataset(observations=observations,
                      actions=actions,
                      rewards=rewards,
                      terminals=dones,
                      timeouts=truncated,
                      action_space=d3rlpy.constants.ActionSpace.CONTINUOUS
                      )


In [ ]:
encoder_factory = d3rlpy.models.VectorEncoderFactory(
    hidden_units=[2000, 2000],
    activation='relu',
)

In [ ]:
from d3rlpy.algos import DQNConfig, SACConfig, IQLConfig, TD3PlusBCConfig, BCConfig, CQLConfig
from d3rlpy.preprocessing import ActionScaler, MinMaxActionScaler

iql = CQLConfig(action_scaler=MinMaxActionScaler(), actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory).create(device="cuda:0")
iql.build_with_dataset(mdp_dataset)

In [ ]:
iql.get_action_type()

In [ ]:
from d3rlpy.metrics import TDErrorEvaluator, DiscountedSumOfAdvantageEvaluator

# calculate metrics with training dataset
td_error_evaluator = TDErrorEvaluator(episodes=mdp_dataset.episodes)
discounted_sum_of_advantage_evaluator = DiscountedSumOfAdvantageEvaluator(episodes=mdp_dataset.episodes)

In [ ]:
iql.fit(
    mdp_dataset,
    n_steps=100000,
    save_interval=1000,
)

In [ ]:
iql.save_model('cql_model_main_v2.pt')